In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import anndata as ad
import numpy as np
import os
import scanpy as sc
import numpy as np

from evaluate import evaluate
from prediction import ADTPredictor, ADTPredictorKRREnsemble

### Train on NeurIPS competition dataset

In [7]:
#dataset_path = "../NeurIPS/competition_data/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_"
dataset_path = "../local/NeurIPS/competition_data/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_"
gex_train = ad.read_h5ad(dataset_path + "train_mod1.h5ad")
gex_test = ad.read_h5ad(dataset_path + "test_mod1.h5ad")
adt_train = ad.read_h5ad(dataset_path + "train_mod2.h5ad")
adt_test = ad.read_h5ad(dataset_path + "test_mod2.h5ad")

In [8]:
# using high-level interface
pipe = ADTPredictor(do_log1p=False)
# fit on training data
# gex_test is optional and is used for transductive preprocessing if provided
# gex_names and adt_names are optional and should refer to the variable names of gex_train and adt_train
# if not provided, the predict() method will assume that all the columns of the test GEX matrix are in the same order as the training GEX matrix
pipe.fit(gex_train=gex_train.X.toarray(), adt_train=adt_train.X.toarray(), gex_test=gex_test.X.toarray(), gex_names=gex_train.var_names.to_numpy(), adt_names=adt_train.var_names.to_numpy())

In [9]:
adt_pred, adt_names = pipe.predict(gex_test.X.toarray())
evaluate(adt_pred, adt_test.X.toarray())

RMSE: 0.38256356
Pearson correlation: 0.8745113684604644
Spearman correlation: 0.8415399498602144


(0.38256356, 0.8745113684604644, 0.8415399498602144)

In [10]:
# adt names are also stored as a property
pipe.adt_names

array(['CD86', 'CD274', 'CD270', 'CD155', 'CD112', 'CD47', 'CD48', 'CD40',
       'CD154', 'CD52', 'CD3', 'CD8', 'CD56', 'CD19', 'CD33', 'CD11c',
       'HLA-A-B-C', 'CD45RA', 'CD123', 'CD7', 'CD105', 'CD49f', 'CD194',
       'CD4', 'CD44', 'CD14', 'CD16', 'CD25', 'CD45RO', 'CD279', 'TIGIT',
       'CD20', 'CD335', 'CD31', 'Podoplanin', 'CD146', 'IgM', 'CD5',
       'CD195', 'CD32', 'CD196', 'CD185', 'CD103', 'CD69', 'CD62L',
       'CD161', 'CD152', 'CD223', 'KLRG1', 'CD27', 'CD107a', 'CD95',
       'CD134', 'HLA-DR', 'CD1c', 'CD11b', 'CD64', 'CD141', 'CD1d',
       'CD314', 'CD35', 'CD57', 'CD272', 'CD278', 'CD58', 'CD39',
       'CX3CR1', 'CD24', 'CD21', 'CD11a', 'CD79b', 'CD244', 'CD169',
       'integrinB7', 'CD268', 'CD42b', 'CD54', 'CD62P', 'CD119', 'TCR',
       'CD192', 'CD122', 'FceRIa', 'CD41', 'CD137', 'CD163', 'CD83',
       'CD124', 'CD13', 'CD2', 'CD226', 'CD29', 'CD303', 'CD49b', 'CD81',
       'IgD', 'CD18', 'CD28', 'CD38', 'CD127', 'CD45', 'CD22', 'CD71',
       'CD26

In [11]:
# save the trained pipeline to a file
pipe.save("../data/ADTPredictor_neuripstrain_alltypes.joblib")

### Train on NeurIPS competition dataset, only T cells

In [15]:
#dataset_with_celltype = "../../../../../PycharmProjects/ModalityPrediction/datasets/post_competition/openproblems_bmmc_cite_complete.h5ad"
dataset_with_celltype = "../local/NeurIPS/annotated_data/openproblems_bmmc_cite_complete.h5ad"
gex_adt_with_celltype = ad.read_h5ad(dataset_with_celltype)

/home/triller/.virtualenvs/scLinear_code/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
gex_adt_with_celltype.obs["cell_type"].value_counts()

cell_type
CD14+ Mono                          21693
CD4+ T activated                     6966
CD4+ T naive                         5897
NK                                   5434
Reticulocyte                         4272
Erythroblast                         4039
Naive CD20+ B IGKC+                  3990
CD8+ T naive                         3107
CD16+ Mono                           2635
NK CD158e1+                          2167
Naive CD20+ B IGKC-                  1979
G/M prog                             1881
pDC                                  1758
HSC                                  1703
cDC2                                 1702
Lymph prog                           1681
Transitional B                       1575
Proerythroblast                      1512
CD8+ T CD57+ CD45RO+                 1470
Normoblast                           1435
CD8+ T CD57+ CD45RA+                 1303
CD8+ T TIGIT+ CD45RO+                1160
CD4+ T activated integrinB7+         1056
CD8+ T TIGIT+ CD45RA+   

In [16]:
def filter_to_Tcells(adata):
    gex_data = adata[:, adata.var["feature_types"] == "GEX"]
    adt_data = adata[:, adata.var["feature_types"] == "ADT"]
    cell_mask = adata.obs["cell_type"].str.contains("T ") | adata.obs["cell_type"].str.endswith("T")
    return gex_data[cell_mask], adt_data[cell_mask]

In [17]:
gex_train_Tcells, adt_train_Tcells = filter_to_Tcells(gex_adt_with_celltype[gex_train.obs_names])
gex_test_Tcells, adt_test_Tcells = filter_to_Tcells(gex_adt_with_celltype[gex_test.obs_names])

In [18]:
gex_train_Tcells.obs["cell_type"].value_counts()

cell_type
CD4+ T activated                    4989
CD4+ T naive                        4294
CD8+ T naive                        2081
CD8+ T TIGIT+ CD45RO+                869
CD8+ T CD57+ CD45RA+                 853
CD8+ T TIGIT+ CD45RA+                781
CD4+ T activated integrinB7+         725
CD8+ T CD49f+                        594
CD8+ T CD69+ CD45RO+                 498
CD8+ T CD69+ CD45RA+                 456
MAIT                                 438
T reg                                393
CD8+ T CD57+ CD45RO+                 380
gdT CD158b+                          210
gdT TCRVD2+                          155
CD4+ T CD314+ CD45RA+                 83
dnT                                   49
CD8+ T naive CD127+ CD26- CD101-      37
T prog cycling                        18
Name: count, dtype: int64

In [20]:
# first evaluate the performance on T cells using the model trained on all cell types
pipe = ADTPredictor(do_log1p=False)
pipe.load("../data/ADTPredictor_neuripstrain_alltypes.joblib")
adt_pred, adt_names = pipe.predict(np.log1p(gex_test_Tcells.X.toarray()))
evaluate(adt_pred, adt_test_Tcells.X.toarray())

RMSE: 0.38099897
Pearson correlation: 0.8675845916677649
Spearman correlation: 0.8400509270626534


(0.38099897, 0.8675845916677649, 0.8400509270626534)

In [21]:
# train a new pipeline on only T cells
pipe_Tcells = ADTPredictor(do_log1p=True)
pipe_Tcells.fit(gex_train=gex_train_Tcells.X.toarray(),
                adt_train=adt_train_Tcells.X.toarray(),
                gex_test=gex_test_Tcells.X.toarray(),
                gex_names=gex_train_Tcells.var_names.to_numpy(),
                adt_names=adt_train_Tcells.var_names.to_numpy())

In [22]:
# evaluate on T cells
adt_pred, adt_names = pipe_Tcells.predict(gex_test_Tcells.X.toarray())
evaluate(adt_pred, adt_test_Tcells.X.toarray())

RMSE: 0.37285337
Pearson correlation: 0.8735118246440898
Spearman correlation: 0.8511490598887181


(0.37285337, 0.8735118246440898, 0.8511490598887181)

In [23]:
# save the trained pipeline
pipe_Tcells.save("../data/ADTPredictor_neuripstrain_Tcells.joblib")

In [29]:
# load the blood tcells dataset, to test the generalization of the model
#datasets_path = "../../../../../PycharmProjects/ModalityPrediction/datasets/"
#gex_path = os.path.join(datasets_path, "bloodTcellsCITEseqDOGMAseq/GSM6032900_CITE_RNA_filtered_feature_bc_matrix.h5")
#adt_path = os.path.join(datasets_path, "bloodTcellsCITEseqDOGMAseq/GSM6032898_CITE_ADT.csv.gz")

datasets_path = "../local/NeurIPS/GSM6032900/"
gex_path = os.path.join(datasets_path, "GSM6032900_CITE_RNA_filtered_feature_bc_matrix.h5")
adt_path = os.path.join(datasets_path, "GSM6032898_CITE_ADT.csv.gz")


print("Loading gex, adt")
gex_tcellsdset = sc.read_10x_h5(gex_path)
adt_tcellsdset = ad.read_csv(adt_path, first_column_names=True).transpose()

print("Filtering duplicate variables, zero variables, zero cells, fixing adt var names")
# filter out duplicate variables
gex_tcellsdset.var_names_make_unique()
adt_tcellsdset.var_names_make_unique()
# filter out cells with no adt measurements
gex_tcellsdset = gex_tcellsdset[adt_tcellsdset.obs_names]
gex_tcellsdset_df = gex_tcellsdset.to_df()
# filter out genes with constant expression in every cell
gex_tcellsdset = gex_tcellsdset[:, (gex_tcellsdset_df != gex_tcellsdset_df.iloc[0]).any(axis=0)]
# delete the last 6 characters of the protein names (to match the protein names in the competition dataset)
adt_tcellsdset.var_names = [x[:-6] for x in adt_tcellsdset.var_names]

Loading gex, adt


/home/triller/.virtualenvs/scLinear_code/lib/python3.9/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Filtering duplicate variables, zero variables, zero cells, fixing adt var names


In [30]:
# predict using the model trained on all cell types
# by providing gex_names, the predict() method will filter out the GEX variables on which the model was not trained
# the GEX variables that were in the training set but not in gex_names will be set to 0
adt_tcellsdset_pred, adt_tcellsdset_pred_names = pipe.predict(np.log1p(gex_tcellsdset.X.toarray()), gex_names=gex_tcellsdset.var_names.to_numpy())

In [31]:
# filter out the proteins that are not in the competition dataset
adt_tcellsdset_true = adt_tcellsdset[:, adt_tcellsdset_pred_names[np.isin(adt_tcellsdset_pred_names, adt_tcellsdset.var_names)]]

In [32]:
evaluate(adt_tcellsdset_pred[:, np.isin(adt_tcellsdset_pred_names, adt_tcellsdset_true.var_names)], adt_tcellsdset_true.X)

RMSE: 48.76078
Pearson correlation: 0.47266432314340695
Spearman correlation: 0.646443954411851


(48.76078, 0.47266432314340695, 0.646443954411851)

In [33]:
# predict using the model trained on T cells
adt_tcellsdset_pred, adt_tcellsdset_pred_names = pipe_Tcells.predict(gex_tcellsdset.X.toarray(), gex_names=gex_tcellsdset.var_names.to_numpy())
evaluate(adt_tcellsdset_pred[:, np.isin(adt_tcellsdset_pred_names, adt_tcellsdset_true.var_names)], adt_tcellsdset_true.X)

RMSE: 48.757954
Pearson correlation: 0.49424800950172715
Spearman correlation: 0.6850092893825263


(48.757954, 0.49424800950172715, 0.6850092893825263)

### Train on NeurIPS competition dataset, only B cells

In [34]:
def filter_to_Bcells(adata):
    gex_data = adata[:, adata.var["feature_types"] == "GEX"]
    adt_data = adata[:, adata.var["feature_types"] == "ADT"]
    cell_mask = adata.obs["cell_type"].str.contains("B ") | adata.obs["cell_type"].str.endswith("B")
    return gex_data[cell_mask], adt_data[cell_mask]

gex_train_Bcells, adt_train_Bcells = filter_to_Bcells(gex_adt_with_celltype[gex_train.obs_names])
gex_test_Bcells, adt_test_Bcells = filter_to_Bcells(gex_adt_with_celltype[gex_test.obs_names])

In [35]:
gex_train_Bcells.obs["cell_type"].value_counts()

cell_type
Naive CD20+ B IGKC+    2464
Naive CD20+ B IGKC-    1205
Transitional B          922
B1 B IGKC+              541
B1 B IGKC-              407
Name: count, dtype: int64

In [36]:
# train a new pipeline on only B cells
pipe_Bcells = ADTPredictor(do_log1p=True)
pipe_Bcells.fit(gex_train=gex_train_Bcells.X.toarray(),
                adt_train=adt_train_Bcells.X.toarray(),
                gex_test=gex_test_Bcells.X.toarray(),
                gex_names=gex_train_Bcells.var_names.to_numpy(),
                adt_names=adt_train_Bcells.var_names.to_numpy())

In [37]:
# evaluate on B cells
adt_pred, adt_names = pipe_Bcells.predict(gex_test_Bcells.X.toarray())
evaluate(adt_pred, adt_test_Bcells.X.toarray())

RMSE: 0.337103
Pearson correlation: 0.8980249009905821
Spearman correlation: 0.8588047383487498


(0.337103, 0.8980249009905821, 0.8588047383487498)

In [38]:
# save the trained pipeline
pipe_Bcells.save("../data/ADTPredictor_neuripstrain_Bcells.joblib")

### Train on NeurIPS competition dataset, only NK cells

In [39]:
def filter_to_NKcells(adata):
    gex_data = adata[:, adata.var["feature_types"] == "GEX"]
    adt_data = adata[:, adata.var["feature_types"] == "ADT"]
    cell_mask = adata.obs["cell_type"].str.contains("NK ") | adata.obs["cell_type"].str.endswith("NK")
    return gex_data[cell_mask], adt_data[cell_mask]

gex_train_NKcells, adt_train_NKcells = filter_to_NKcells(gex_adt_with_celltype[gex_train.obs_names])
gex_test_NKcells, adt_test_NKcells = filter_to_NKcells(gex_adt_with_celltype[gex_test.obs_names])

In [40]:
gex_train_NKcells.obs["cell_type"].value_counts()

cell_type
NK             3657
NK CD158e1+    1683
Name: count, dtype: int64

In [41]:
# train a new pipeline on only NK cells
pipe_NKcells = ADTPredictor(do_log1p=True)
pipe_NKcells.fit(gex_train=gex_train_NKcells.X.toarray(),
                adt_train=adt_train_NKcells.X.toarray(),
                gex_test=gex_test_NKcells.X.toarray(),
                gex_names=gex_train_NKcells.var_names.to_numpy(),
                adt_names=adt_train_NKcells.var_names.to_numpy())

In [42]:
# evaluate on NK cells
adt_pred, adt_names = pipe_NKcells.predict(gex_test_NKcells.X.toarray())
evaluate(adt_pred, adt_test_NKcells.X.toarray())

RMSE: 0.3473118
Pearson correlation: 0.8837162800589263
Spearman correlation: 0.8567011868914186


(0.3473118, 0.8837162800589263, 0.8567011868914186)

In [43]:
# save the trained pipeline
pipe_NKcells.save("../data/ADTPredictor_neuripstrain_NKcells.joblib")

### Load the pretrained kernel ridge regression ensemble model

In [44]:
pipe = ADTPredictorKRREnsemble(do_log1p=False)
pipe.load('../data/ADTPredictorKRREnsemble_neuripstrain_alltypes.joblib')
adt_pred, adt_names = pipe.predict(gex_test.X.toarray())
evaluate(adt_pred, adt_test.X.toarray())

FileNotFoundError: [Errno 2] No such file or directory: 'ADTPredictorKRREnsemble_neuripstrain_alltypes.joblib'